# EDA 시각화 코드
결측치 및 이상치 탐색

In [ ]:
import matplotlib

matplotlib.use("Agg")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import font_manager, rc
import regex
import warnings

warnings.filterwarnings("ignore")

## 결측치 및 이상치 시각화 클래스

In [10]:
class EDAVisualizer:

    def __init__(self, data_path):
        self.data_path = data_path
        self.users = None
        self.books = None
        self.train = None
        self.test = None

    def load_data(self):
        print("> 데이터 로딩 중...")
        self.users = pd.read_csv(self.data_path + "users.csv")
        self.books = pd.read_csv(self.data_path + "books.csv")
        self.train = pd.read_csv(self.data_path + "train_ratings.csv")
        self.test = pd.read_csv(self.data_path + "test_ratings.csv")
        print("> 데이터 로딩 완료\n")

    def plot_missing_values(self, save_path="eda_missing_values.png"):
        fig, axes = plt.subplots(1, 2, figsize=(16, 6))

        # [Users] 결측치
        users_missing = self.users.isnull().sum()
        users_missing_pct = (users_missing / len(self.users) * 100).sort_values(
            ascending=False
        )

        ax1 = axes[0]
        users_missing_pct.plot(kind="barh", ax=ax1, color="coral")
        ax1.set_title("Users Dataset - Missing Values (%)", fontsize=14, weight="bold")
        ax1.set_xlabel("Missing Percentage (%)")
        for i, v in enumerate(users_missing_pct):
            ax1.text(v + 0.5, i, f"{v:.1f}%", va="center")

        # [Books] 결측치
        books_missing = self.books.isnull().sum()
        books_missing_pct = (books_missing / len(self.books) * 100).sort_values(
            ascending=False
        )

        ax2 = axes[1]
        books_missing_pct.plot(kind="barh", ax=ax2, color="skyblue")
        ax2.set_title("Books Dataset - Missing Values (%)", fontsize=14, weight="bold")
        ax2.set_xlabel("Missing Percentage (%)")
        for i, v in enumerate(books_missing_pct):
            ax2.text(v + 0.5, i, f"{v:.1f}%", va="center")

        plt.tight_layout()
        plt.savefig(save_path, dpi=300, bbox_inches="tight")
        print(f"💾 결측치 시각화 저장: {save_path}")
        plt.show()

    # 나이 분포 이상치 탐지
    def plot_age_distribution(self, save_path="eda_age_distribution.png"):
        fig, axes = plt.subplots(2, 2, figsize=(16, 10))

        age_data = self.users["age"].dropna()

        # 1. 전체 분포
        ax1 = axes[0, 0]
        ax1.hist(age_data, bins=50, color="lightblue", edgecolor="black", alpha=0.7)
        ax1.set_title("Age Distribution (All Data)", fontsize=14, weight="bold")
        ax1.set_xlabel("Age")
        ax1.set_ylabel("Frequency")
        ax1.axvline(
            age_data.mean(),
            color="red",
            linestyle="--",
            label=f"Mean: {age_data.mean():.1f}",
        )
        ax1.axvline(
            age_data.median(),
            color="green",
            linestyle="--",
            label=f"Median: {age_data.median():.1f}",
        )
        ax1.legend()

        # 2. 박스 플롯 (이상치 확인)
        ax2 = axes[0, 1]
        ax2.boxplot(
            age_data,
            vert=True,
            patch_artist=True,
            boxprops=dict(facecolor="lightgreen", alpha=0.7),
        )
        ax2.set_title("Age Boxplot - Outlier Detection", fontsize=14, weight="bold")
        ax2.set_ylabel("Age")

        # IQR 기반 이상치 경계선 표시
        Q1, Q3 = age_data.quantile([0.25, 0.75])
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        ax2.axhline(
            lower_bound,
            color="red",
            linestyle="--",
            alpha=0.5,
            label=f"Lower: {lower_bound:.1f}",
        )
        ax2.axhline(
            upper_bound,
            color="red",
            linestyle="--",
            alpha=0.5,
            label=f"Upper: {upper_bound:.1f}",
        )
        ax2.legend()

        # 3. 합리적 범위 분포 (0-120세)
        ax3 = axes[1, 0]
        reasonable_age = age_data[(age_data >= 0) & (age_data <= 120)]
        ax3.hist(reasonable_age, bins=50, color="gold", edgecolor="black", alpha=0.7)
        ax3.set_title(
            f"Reasonable Age (0-120) - {len(reasonable_age)} records",
            fontsize=14,
            weight="bold",
        )
        ax3.set_xlabel("Age")
        ax3.set_ylabel("Frequency")

        # 4. 이상치 분석 테이블
        ax4 = axes[1, 1]
        ax4.axis("off")

        outliers = age_data[(age_data < lower_bound) | (age_data > upper_bound)]
        stats_data = [
            ["Total Records", f"{len(self.users):,}"],
            [
                "Missing Age",
                f'{self.users["age"].isnull().sum():,} ({self.users["age"].isnull().sum()/len(self.users)*100:.1f}%)',
            ],
            ["Valid Age", f"{len(age_data):,}"],
            ["Mean Age", f"{age_data.mean():.2f}"],
            ["Median Age", f"{age_data.median():.2f}"],
            ["Min Age", f"{age_data.min():.2f}"],
            ["Max Age", f"{age_data.max():.2f}"],
            [
                "IQR Outliers",
                f"{len(outliers):,} ({len(outliers)/len(age_data)*100:.1f}%)",
            ],
            ["Age < 0", f"{(age_data < 0).sum()}"],
            ["Age > 120", f"{(age_data > 120).sum()}"],
        ]

        table = ax4.table(
            cellText=stats_data,
            colLabels=["Metric", "Value"],
            cellLoc="left",
            loc="center",
            colWidths=[0.6, 0.4],
        )
        table.auto_set_font_size(False)
        table.set_fontsize(10)
        table.scale(1, 2)
        ax4.set_title("Age Statistics Summary", fontsize=14, weight="bold", pad=20)

        plt.tight_layout()
        plt.savefig(save_path, dpi=300, bbox_inches="tight")
        print(f"💾 나이 분포 저장: {save_path}")
        plt.show()

    # 출판연도 분포 및 이상치 탐지
    def plot_year_distribution(self, save_path="eda_year_distribution.png"):
        fig, axes = plt.subplots(2, 2, figsize=(16, 10))

        year_data = self.books["year_of_publication"].dropna()

        # 1. 전체 분포
        ax1 = axes[0, 0]
        ax1.hist(year_data, bins=100, color="lightcoral", edgecolor="black", alpha=0.7)
        ax1.set_title(
            "Publication Year Distribution (All Data)", fontsize=14, weight="bold"
        )
        ax1.set_xlabel("Year")
        ax1.set_ylabel("Frequency")
        ax1.axvline(
            year_data.median(),
            color="green",
            linestyle="--",
            label=f"Median: {year_data.median():.0f}",
        )
        ax1.legend()

        # 2. 박스플롯
        ax2 = axes[0, 1]
        ax2.boxplot(
            year_data,
            vert=True,
            patch_artist=True,
            boxprops=dict(facecolor="lightblue", alpha=0.7),
        )
        ax2.set_title("Year Boxplot - Outlier Detection", fontsize=14, weight="bold")
        ax2.set_ylabel("Year")

        # 3. 합리적 범위 (1900-2025) 로 임의 설정
        ax3 = axes[1, 0]
        reasonable_year = year_data[(year_data >= 1900) & (year_data <= 2025)]
        ax3.hist(
            reasonable_year,
            bins=50,
            color="mediumseagreen",
            edgecolor="black",
            alpha=0.7,
        )
        ax3.set_title(
            f"Reasonable Year (1900-2025) - {len(reasonable_year)} records",
            fontsize=14,
            weight="bold",
        )
        ax3.set_xlabel("Year")
        ax3.set_ylabel("Frequency")

        # 4. 통계 테이블
        ax4 = axes[1, 1]
        ax4.axis("off")

        stats_data = [
            ["Total Books", f"{len(self.books):,}"],
            ["Valid Years", f"{len(year_data):,}"],
            ["Mean Year", f"{year_data.mean():.0f}"],
            ["Median Year", f"{year_data.median():.0f}"],
            ["Min Year", f"{year_data.min():.0f}"],
            ["Max Year", f"{year_data.max():.0f}"],
            ["Year < 1900", f"{(year_data < 1900).sum():,}"],
            ["Year > 2025", f"{(year_data > 2025).sum():,}"],
            [
                "1900-2025 Range",
                f"{len(reasonable_year):,} ({len(reasonable_year)/len(year_data)*100:.1f}%)",
            ],
        ]

        table = ax4.table(
            cellText=stats_data,
            colLabels=["Metric", "Value"],
            cellLoc="left",
            loc="center",
            colWidths=[0.6, 0.4],
        )
        table.auto_set_font_size(False)
        table.set_fontsize(10)
        table.scale(1, 2)
        ax4.set_title("Publication Year Statistics", fontsize=14, weight="bold", pad=20)

        plt.tight_layout()
        plt.savefig(save_path, dpi=300, bbox_inches="tight")
        print(f"💾 출판연도 분포 저장: {save_path}")
        plt.show()

    # 평점 분포
    def plot_rating_distribution(self, save_path="eda_rating_distribution.png"):
        fig, axes = plt.subplots(1, 3, figsize=(18, 5))

        # 1. Train Rating 분포
        ax1 = axes[0]
        rating_counts = self.train["rating"].value_counts().sort_index()
        rating_counts.plot(kind="bar", ax=ax1, color="steelblue", alpha=0.8)
        ax1.set_title("Train Rating Distribution", fontsize=14, weight="bold")
        ax1.set_xlabel("Rating")
        ax1.set_ylabel("Frequency")
        ax1.set_xticklabels(ax1.get_xticklabels(), rotation=0)
        for i, v in enumerate(rating_counts):
            ax1.text(i, v + 500, str(v), ha="center", fontsize=9)

        # 2. Rating 박스플롯
        ax2 = axes[1]
        ax2.boxplot(
            self.train["rating"],
            vert=True,
            patch_artist=True,
            boxprops=dict(facecolor="lightgreen", alpha=0.7),
        )
        ax2.set_title("Rating Boxplot", fontsize=14, weight="bold")
        ax2.set_ylabel("Rating")
        ax2.set_ylim(0, 11)

        # 3. 통계 테이블
        ax3 = axes[2]
        ax3.axis("off")

        stats_data = [
            ["Total Ratings", f"{len(self.train):,}"],
            ["Mean Rating", f'{self.train["rating"].mean():.2f}'],
            ["Median Rating", f'{self.train["rating"].median():.1f}'],
            ["Mode Rating", f'{self.train["rating"].mode()[0]}'],
            ["Min Rating", f'{self.train["rating"].min()}'],
            ["Max Rating", f'{self.train["rating"].max()}'],
            ["Std Dev", f'{self.train["rating"].std():.2f}'],
            [
                "Outliers (<1 or >10)",
                f'{((self.train["rating"] < 1) | (self.train["rating"] > 10)).sum()}',
            ],
        ]

        table = ax3.table(
            cellText=stats_data,
            colLabels=["Metric", "Value"],
            cellLoc="left",
            loc="center",
            colWidths=[0.6, 0.4],
        )
        table.auto_set_font_size(False)
        table.set_fontsize(10)
        table.scale(1, 2)
        ax3.set_title("Rating Statistics", fontsize=14, weight="bold", pad=20)

        plt.tight_layout()
        plt.savefig(save_path, dpi=300, bbox_inches="tight")
        print(f"💾 평점 분포 저장: {save_path}")
        plt.show()

    # 위치 분석
    def plot_location_analysis(self, save_path="eda_location_analysis.png"):
        fig, axes = plt.subplots(2, 2, figsize=(16, 10))

        def split_location(x):
            res = x.split(",")
            res = [i.strip().lower() for i in res]
            res = [regex.sub(r"[^a-zA-Z/ ]", "", i) for i in res]
            res = [i if i not in ["n/a", ""] else np.nan for i in res]
            res.reverse()
            return res

        # Location 파싱
        self.users["loc_parsed"] = self.users["location"].apply(split_location)
        self.users["loc_country"] = self.users["loc_parsed"].apply(
            lambda x: x[0] if len(x) > 0 else np.nan
        )
        self.users["loc_state"] = self.users["loc_parsed"].apply(
            lambda x: x[1] if len(x) > 1 else np.nan
        )
        self.users["loc_city"] = self.users["loc_parsed"].apply(
            lambda x: x[2] if len(x) > 2 else np.nan
        )

        # 1. Location 결측치
        ax1 = axes[0, 0]
        loc_missing = pd.Series(
            {
                "Country": self.users["loc_country"].isnull().sum(),
                "State": self.users["loc_state"].isnull().sum(),
                "City": self.users["loc_city"].isnull().sum(),
            }
        )
        loc_missing_pct = loc_missing / len(self.users) * 100
        loc_missing_pct.plot(kind="bar", ax=ax1, color=["coral", "gold", "skyblue"])
        ax1.set_title("Location Missing Values (%)", fontsize=14, weight="bold")
        ax1.set_ylabel("Missing Percentage (%)")
        ax1.set_xticklabels(ax1.get_xticklabels(), rotation=0)
        for i, v in enumerate(loc_missing_pct):
            ax1.text(i, v + 1, f"{v:.1f}%", ha="center")

        # 2. Top 10 Countries
        ax2 = axes[0, 1]
        top_countries = self.users["loc_country"].value_counts().head(10)
        top_countries.plot(kind="barh", ax=ax2, color="mediumseagreen")
        ax2.set_title("Top 10 Countries", fontsize=14, weight="bold")
        ax2.set_xlabel("User Count")
        for i, v in enumerate(top_countries):
            ax2.text(v + 100, i, str(v), va="center")

        # 3. n/a 패턴 분석
        ax3 = axes[1, 0]
        na_patterns = self.users["location"].value_counts().head(10)
        na_patterns.plot(kind="barh", ax=ax3, color="lightcoral")
        ax3.set_title("Top 10 Location Patterns", fontsize=14, weight="bold")
        ax3.set_xlabel("Count")

        # 4. 통계 테이블
        ax4 = axes[1, 1]
        ax4.axis("off")

        stats_data = [
            ["Total Users", f"{len(self.users):,}"],
            ["Unique Countries", f'{self.users["loc_country"].nunique()}'],
            ["Unique States", f'{self.users["loc_state"].nunique()}'],
            ["Unique Cities", f'{self.users["loc_city"].nunique()}'],
            [
                "Missing Country",
                f'{loc_missing["Country"]:,} ({loc_missing_pct["Country"]:.1f}%)',
            ],
            [
                "Missing State",
                f'{loc_missing["State"]:,} ({loc_missing_pct["State"]:.1f}%)',
            ],
            [
                "Missing City",
                f'{loc_missing["City"]:,} ({loc_missing_pct["City"]:.1f}%)',
            ],
            [
                '"n/a, n/a, n/a"',
                f'{(self.users["location"] == "n/a, n/a, n/a").sum():,}',
            ],
        ]

        table = ax4.table(
            cellText=stats_data,
            colLabels=["Metric", "Value"],
            cellLoc="left",
            loc="center",
            colWidths=[0.6, 0.4],
        )
        table.auto_set_font_size(False)
        table.set_fontsize(10)
        table.scale(1, 2)
        ax4.set_title("Location Statistics", fontsize=14, weight="bold", pad=20)

        plt.tight_layout()
        plt.savefig(save_path, dpi=300, bbox_inches="tight")
        print(f"💾 Location 분석 저장: {save_path}")
        plt.show()

    # User-Item 인터랙션 분석
    # for [Cold Start 문제 파악]
    def plot_user_item_interaction(self, save_path="eda_user_item_stats.png"):
        fig, axes = plt.subplots(2, 2, figsize=(16, 10))

        # User별 평점 개수
        user_rating_counts = self.train.groupby("user_id").size()

        # Item별 평점 개수
        item_rating_counts = self.train.groupby("isbn").size()

        # 1. User Rating Count 분포
        ax1 = axes[0, 0]
        ax1.hist(
            user_rating_counts, bins=50, color="steelblue", edgecolor="black", alpha=0.7
        )
        ax1.set_title("User Rating Count Distribution", fontsize=14, weight="bold")
        ax1.set_xlabel("Number of Ratings per User")
        ax1.set_ylabel("Frequency")
        ax1.axvline(
            user_rating_counts.median(),
            color="red",
            linestyle="--",
            label=f"Median: {user_rating_counts.median():.0f}",
        )
        ax1.legend()

        # 2. Item Rating Count 분포
        ax2 = axes[0, 1]
        ax2.hist(
            item_rating_counts, bins=50, color="coral", edgecolor="black", alpha=0.7
        )
        ax2.set_title("Book Rating Count Distribution", fontsize=14, weight="bold")
        ax2.set_xlabel("Number of Ratings per Book")
        ax2.set_ylabel("Frequency")
        ax2.axvline(
            item_rating_counts.median(),
            color="red",
            linestyle="--",
            label=f"Median: {item_rating_counts.median():.0f}",
        )
        ax2.legend()

        # 3. Cold Start 문제 (평점 1개 이하)
        ax3 = axes[1, 0]
        cold_start_data = pd.DataFrame(
            {
                "Users (≤5 ratings)": [
                    (user_rating_counts <= i).sum() for i in [1, 3, 5]
                ],
                "Books (≤5 ratings)": [
                    (item_rating_counts <= i).sum() for i in [1, 3, 5]
                ],
            },
            index=["≤1", "≤3", "≤5"],
        )

        cold_start_data.plot(kind="bar", ax=ax3, color=["steelblue", "coral"])
        ax3.set_title("Cold Start Problem Analysis", fontsize=14, weight="bold")
        ax3.set_ylabel("Count")
        ax3.set_xticklabels(ax3.get_xticklabels(), rotation=0)
        ax3.legend()

        # 4. 통계 테이블
        ax4 = axes[1, 1]
        ax4.axis("off")

        stats_data = [
            ["Total Users", f'{self.train["user_id"].nunique():,}'],
            ["Total Books", f'{self.train["isbn"].nunique():,}'],
            ["Total Ratings", f"{len(self.train):,}"],
            ["Avg Ratings/User", f"{user_rating_counts.mean():.2f}"],
            ["Avg Ratings/Book", f"{item_rating_counts.mean():.2f}"],
            ["Users w/ 1 rating", f"{(user_rating_counts == 1).sum():,}"],
            ["Books w/ 1 rating", f"{(item_rating_counts == 1).sum():,}"],
            [
                "Sparsity",
                f'{(1 - len(self.train) / (self.train["user_id"].nunique() * self.train["isbn"].nunique())) * 100:.2f}%',
            ],
        ]

        table = ax4.table(
            cellText=stats_data,
            colLabels=["Metric", "Value"],
            cellLoc="left",
            loc="center",
            colWidths=[0.6, 0.4],
        )
        table.auto_set_font_size(False)
        table.set_fontsize(10)
        table.scale(1, 2)
        ax4.set_title("User-Item Statistics", fontsize=14, weight="bold", pad=20)

        plt.tight_layout()
        plt.savefig(save_path, dpi=300, bbox_inches="tight")
        print(f"💾 User-Item 인터랙션 저장: {save_path}")
        plt.show()

    # 전체 EDA 리포트 생성
    def generate_full_report(self):
        print("=" * 60)
        print("EDA 시각화 시작")
        print("=" * 60)

        self.load_data()

        print("\n1) 결측치 분석 중...")
        self.plot_missing_values()

        print("\n2) 나이 분포 분석 중...")
        self.plot_age_distribution()

        print("\n3) 출판연도 분포 분석 중...")
        self.plot_year_distribution()

        print("\n4) 평점 분포 분석 중...")
        self.plot_rating_distribution()

        print("\n5) 위치 분석 중...")
        self.plot_location_analysis()

        print("\n6) User-Item 인터랙션 분석 중...")
        self.plot_user_item_interaction()

        print("\n" + "=" * 60)
        print("EDA 시각화 완료")
        print("=" * 60)

## 사용

In [11]:
if __name__ == "__main__":
    DATA_PATH = "/data/ephemeral/home/sojin/data/"

    visualizer = EDAVisualizer(DATA_PATH)
    visualizer.generate_full_report()

EDA 시각화 시작
> 데이터 로딩 중...
> 데이터 로딩 완료


1) 결측치 분석 중...
💾 결측치 시각화 저장: eda_missing_values.png

2) 나이 분포 분석 중...
💾 나이 분포 저장: eda_age_distribution.png

3) 출판연도 분포 분석 중...
💾 출판연도 분포 저장: eda_year_distribution.png

4) 평점 분포 분석 중...
💾 평점 분포 저장: eda_rating_distribution.png

5) 위치 분석 중...
💾 Location 분석 저장: eda_location_analysis.png

6) User-Item 인터랙션 분석 중...
💾 User-Item 인터랙션 저장: eda_user_item_stats.png

EDA 시각화 완료
